In [1]:
# import tensorflow as tf
# if tf.__version__ != "2.14.0":
#     print(f"Current TensorFlow version: {tf.__version__}, switching to 2.14.0")

#     # Uninstall current TensorFlow version
#     !pip uninstall -y tensorflow

#     # Install TensorFlow 2.10
#     !pip install numpy==1.26 --force-reinstall
#     !pip install tensorflow==2.14.0


#     # After installation, restart runtime
#     print("TensorFlow 2.14.0 installed.")
#     print("Please click on the Runtime > Restart session and run all.")
# else:
#     print("TensorFlow 2.14.0 is already installed.")

In [2]:
!pip install ml_dtypes>=0.5.1
!pip install tensorflow==2.19.0
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

In [3]:
l0 = Dense(units=1, input_shape=[1])
model = Sequential([l0])
model.compile(optimizer='sgd', loss='mean_squared_error')

xs = np.array([-1.0, 0.0, 1.0, 2.0, 3.0, 4.0], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)

model.fit(xs, ys, epochs=500)

print(model.predict(np.array([10.0])))
print("Here is what I learned: {}".format(l0.get_weights()))

Epoch 1/500


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 593ms/step - loss: 7.4145
Epoch 2/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step - loss: 6.0267
Epoch 3/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step - loss: 4.9309
Epoch 4/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 124ms/step - loss: 4.0649
Epoch 5/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step - loss: 3.3798
Epoch 6/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step - loss: 2.8370
Epoch 7/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - loss: 2.4064
Epoch 8/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 2.0640
Epoch 9/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 1.7910
Epoch 10/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 1.5729
Epoch 11/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 1.3979
Epoch 12/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 1.2570
Epoch 13/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 1.1428
Epoch 14/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 1.0499
Epoch 15/500
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.9736
Epoch 16/500
1/1 ━━━━━━━━

In [4]:
# export_dir = 'saved_model/1'
# tf.saved_model.save(model, export_dir)

import os
import tensorflow as tf

export_dir = 'saved_model/my_model.keras'

# Create the directory if it doesn't exist
os.makedirs(os.path.dirname(export_dir), exist_ok=True)

# Now save the model
model.save(export_dir)
print(f"Model saved successfully to {export_dir}")

Model saved successfully to saved_model/my_model.keras


In [7]:
# Convert the model.
# converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
# tflite_model = converter.convert()
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
print("TFLite_model Completed")

Saved artifact at '/tmp/tmpvevuxlr8'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  137992997161680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137992997162832: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite_model Completed


In [8]:
import pathlib
tflite_model_file = pathlib.Path('model.tflite')
tflite_model_file.write_bytes(tflite_model)

1104

In [9]:
# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_content=tflite_model)
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print(input_details)
print(output_details)

[{'name': 'serving_default_keras_tensor:0', 'index': 0, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]
[{'name': 'StatefulPartitionedCall_1:0', 'index': 3, 'shape': array([1, 1], dtype=int32), 'shape_signature': array([-1,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}]


/usr/local/lib/python3.12/dist-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [10]:
to_predict = np.array([[10.0]], dtype=np.float32)
print(to_predict)
interpreter.set_tensor(input_details[0]['index'], to_predict)
interpreter.invoke()
tflite_results = interpreter.get_tensor(output_details[0]['index'])
print(tflite_results)

[[10.]]
[[18.983522]]
